In [2]:
import pandas as pd
import obonet
import networkx as nx

In [3]:
def get_gene_df():
    gene2hpo = {}
    alt_id = {}
    graph = obonet.read_obo(
        "https://raw.githubusercontent.com/obophenotype/human-phenotype-ontology/master/hp.obo"
    )
    for id_, node in graph.nodes(data=True):
            for alt in node.get('alt_id', []):
                alt_id[alt] = id_
    
    associations_url = "http://purl.obolibrary.org/obo/hp/hpoa/genes_to_phenotype.txt"
    associations = pd.read_csv(associations_url, sep='\t')
    associations = associations[['gene_symbol','hpo_id']]

    gene2hpo = dict()
    for ind in associations.index:
      gene_name = associations["gene_symbol"][ind]
      hp_term = associations["hpo_id"][ind]
      if gene_name not in gene2hpo:
          gene2hpo[gene_name] = list([])
      gene2hpo[gene_name].append(hp_term)
    #gene2hpo[gene_name] = gene2hpo[gene_name] + list(nx.neighbors(graph, hp_term))
    gene2hpo[gene_name] = gene2hpo[gene_name] + list(nx.predecessor(graph, hp_term))
    #gene2hpo[gene_name] = gene2hpo[gene_name] + list(graph.predecessors(hp_term)) + list(graph.successors(hp_term))
    gene2hpo[gene_name] = list(
        dict.fromkeys(gene2hpo[gene_name])
    )  # check for duplicate values and drop them
    gene2hpo[gene_name] = list(
        filter(None, gene2hpo[gene_name])
    )  # check for null values and drop them
    # gene2hpo[gene_name] = comparison.parse_terms(gene2hpo[gene_name])

    gene2hpo = {"Genes": list(gene2hpo.keys()), "HPOs": list(gene2hpo.values())}
    gene2hpo = pd.DataFrame(gene2hpo)
    return gene2hpo, graph, alt_id


In [4]:
def parse_terms(alt_terms, in_terms):
        """ This function parses through terms that are provided, then it returns a new list with updated HPO terms. """
        new_terms = []
        for term in in_terms:
            if term in list(alt_terms.keys()):
                new_terms.append(alt_terms[term])
            else:
                new_terms.append(term)
        return new_terms

In [5]:
print("Generating Gene-Phene vectors from HPO...")
gene_2_hpo, graph, alt_id = get_gene_df()
print("done!")

Generating Gene-Phene vectors from HPO...
done!


In [6]:
input_phenotypes = ["HP:0000639","HP:0001300","HP:0007240","HP:0001310","HP:0007221","HP:0001332","HP:0001337"]
ranked_genes = gene_2_hpo
input_phenotypes = parse_terms(alt_id, input_phenotypes)
input_phenotypes = list(dict.fromkeys(input_phenotypes))  # check for duplicate values and drop them
term_len = len(input_phenotypes)
ranked_genes["Hazel"] = [(len(list(set(input_phenotypes) & set(i)))) / term_len for i in ranked_genes["HPOs"].values]
ranked_genes = (ranked_genes.sort_values(by="Hazel", ascending=False).reset_index(drop=True)
)
#ranked_genes = ranked_genes.drop_duplicates()
print(ranked_genes.shape)
ranked_genes.head()

(5005, 3)


,Genes,HPOs,Hazel
0,POLG,"[HP:0001155, HP:0002495, HP:0002495, HP:000246...",0.857143
1,SQSTM1,"[HP:0002493, HP:0002465, HP:0002463, HP:000246...",0.714286
2,CACNA1A,"[HP:0002486, HP:0002483, HP:0001152, HP:000112...",0.714286
3,TWNK,"[HP:0002495, HP:0002460, HP:0007302, HP:000861...",0.714286
4,STUB1,"[HP:0001181, HP:0001152, HP:0001166, HP:000110...",0.714286


In [7]:
ranked_genes.head(20)

,Genes,HPOs,Hazel
0,POLG,"[HP:0001155, HP:0002495, HP:0002495, HP:000246...",0.857143
1,SQSTM1,"[HP:0002493, HP:0002465, HP:0002463, HP:000246...",0.714286
2,CACNA1A,"[HP:0002486, HP:0002483, HP:0001152, HP:000112...",0.714286
3,TWNK,"[HP:0002495, HP:0002460, HP:0007302, HP:000861...",0.714286
4,STUB1,"[HP:0001181, HP:0001152, HP:0001166, HP:000110...",0.714286
5,ATP13A2,"[HP:0002478, HP:0001167, HP:0002495, HP:000249...",0.714286
6,WARS2,"[HP:0002487, HP:0002474, HP:0002451, HP:002516...",0.714286
7,SCN2A,"[HP:0002487, HP:0025101, HP:0007270, HP:002022...",0.714286
8,ATXN2,"[HP:0001151, HP:0002495, HP:0007311, HP:000374...",0.714286
9,PLA2G6,"[HP:0002483, HP:0002454, HP:0007256, HP:000725...",0.714286


In [13]:
ranked_genes[ranked_genes["Genes"]=="PARK2"]

,Genes,HPOs,Hazel


In [6]:
df = pd.read_csv("../../data/interim/UDN249098.csv.gz")
df.head()

/var/folders/x5/1jk7xryd0rg4d710y586kmvh0000gp/T/ipykernel_53858/61056071.py:1: DtypeWarning: Columns (3,4,9,10,12,13,14,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../../data/interim/UDN249098.csv.gz")


,transcript,gene,consequence,protein_hgvs,cdna_hgvs,chrom,pos,ref_base,alt_base,clingen.disease,clingen.classification,clinvar.id,clinvar.sig,clinvar.rev_stat,clinvar.sig_conf,ncbigene.entrez,omim.omim_id,uniprot.acc,dbsnp.rsid,DITTO
0,NaN,NaN,NaN,NaN,NaN,chr1,10147,C,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs779258992,0.0
1,NaN,NaN,NaN,NaN,NaN,chr1,13868,A,G,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs796086906,0.0
2,NaN,NaN,NaN,NaN,NaN,chr1,14210,G,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs1449130326,0.0
3,NaN,NaN,NaN,NaN,NaN,chr1,14464,A,T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs546169444,0.0
4,NaN,NaN,NaN,NaN,NaN,chr1,15045,C,T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs1172912582,0.0


In [7]:
ditto = df[df["DITTO"] > 0.8]
print(ditto.shape)
ditto.head()

(6978, 20)


,transcript,gene,consequence,protein_hgvs,cdna_hgvs,chrom,pos,ref_base,alt_base,clingen.disease,clingen.classification,clinvar.id,clinvar.sig,clinvar.rev_stat,clinvar.sig_conf,ncbigene.entrez,omim.omim_id,uniprot.acc,dbsnp.rsid,DITTO
108,ENST00000335137,OR4F5,2kb_upstream_variant,NaN,c.-195G>A,chr1,68896,G,A,NaN,NaN,NaN,NaN,NaN,NaN,79501.0,NaN,Q8NH21,rs763813111,0.999976
109,ENST00000641515,OR4F5,intron_variant,NaN,c.10-141G>A,chr1,68896,G,A,NaN,NaN,NaN,NaN,NaN,NaN,79501.0,NaN,Q8NH21,rs763813111,0.906543
830,ENST00000332831,OR4F16,2kb_upstream_variant,NaN,c.-658T>C,chr1,687312,A,G,NaN,NaN,NaN,NaN,NaN,NaN,81399.0,NaN,Q6IEY1,rs1484917993,0.999996
7852,ENST00000378821,TMEM88B,2kb_downstream_variant,NaN,c.*1573del,chr1,1429348,A,-,NaN,NaN,NaN,NaN,NaN,NaN,643965.0,NaN,A6NKF7,NaN,1.000000
7853,ENST00000417917,LINC01770,"2kb_downstream_variant,lnc_RNA",NaN,NaN,chr1,1429348,A,-,NaN,NaN,NaN,NaN,NaN,NaN,643965.0,NaN,A6NKF7,NaN,1.000000


In [8]:
ditto = ditto.merge(ranked_genes, left_on="gene", right_on="Genes", how="left")
ditto = ditto.sort_values(by="Hazel", ascending=False).reset_index(drop=True)
ditto.head()

,transcript,gene,consequence,protein_hgvs,cdna_hgvs,chrom,pos,ref_base,alt_base,clingen.disease,...,clinvar.rev_stat,clinvar.sig_conf,ncbigene.entrez,omim.omim_id,uniprot.acc,dbsnp.rsid,DITTO,Genes,HPOs,Hazel
0,ENST00000417303,PLA2G6,"2kb_upstream_variant,processed_transcript",NaN,NaN,chr22,38193569,AA,-,NaN,...,NaN,NaN,8398.0,NaN,O60733,NaN,0.909572,PLA2G6,"[HP:0002483, HP:0002454, HP:0007256, HP:000725...",0.292683
1,ENST00000536224,GCH1,intron_variant,NaN,c.344-15065_344-15051del,chr14,54880487,ATATATATATACTCC,-,NaN,...,NaN,NaN,2643.0,NaN,P30793,rs1161730704,0.999999,GCH1,"[HP:0002487, HP:0003785, HP:0003781, HP:000245...",0.268293
2,ENST00000622544,GCH1,intron_variant,NaN,c.343+13804dup,chr14,54888517,-,T,NaN,...,NaN,NaN,2643.0,NaN,P30793,NaN,0.943606,GCH1,"[HP:0002487, HP:0003785, HP:0003781, HP:000245...",0.268293
3,ENST00000536224,GCH1,intron_variant,NaN,c.343+5310dup,chr14,54897011,-,T,NaN,...,NaN,NaN,2643.0,NaN,P30793,NaN,0.999998,GCH1,"[HP:0002487, HP:0003785, HP:0003781, HP:000245...",0.268293
4,ENST00000491895,GCH1,intron_variant,NaN,c.343+5310dup,chr14,54897011,-,T,NaN,...,NaN,NaN,2643.0,NaN,P30793,NaN,0.999998,GCH1,"[HP:0002487, HP:0003785, HP:0003781, HP:000245...",0.268293


In [9]:
ditto = ditto.sort_values('DITTO', ascending=False).drop_duplicates(['gene','pos']).sort_index()
print(ditto.shape)
ditto.head()

(2763, 23)


,transcript,gene,consequence,protein_hgvs,cdna_hgvs,chrom,pos,ref_base,alt_base,clingen.disease,...,clinvar.rev_stat,clinvar.sig_conf,ncbigene.entrez,omim.omim_id,uniprot.acc,dbsnp.rsid,DITTO,Genes,HPOs,Hazel
0,ENST00000417303,PLA2G6,"2kb_upstream_variant,processed_transcript",NaN,NaN,chr22,38193569,AA,-,NaN,...,NaN,NaN,8398.0,NaN,O60733,NaN,0.909572,PLA2G6,"[HP:0002483, HP:0002454, HP:0007256, HP:000725...",0.292683
2,ENST00000622544,GCH1,intron_variant,NaN,c.343+13804dup,chr14,54888517,-,T,NaN,...,NaN,NaN,2643.0,NaN,P30793,NaN,0.943606,GCH1,"[HP:0002487, HP:0003785, HP:0003781, HP:000245...",0.268293
7,ENST00000542287,ATXN2,5_prime_UTR_variant,NaN,c.-65+13772_-65+13773del,chr12,111585802,AA,-,NaN,...,NaN,NaN,6311.0,NaN,Q99700,NaN,0.808162,ATXN2,"[HP:0001151, HP:0002495, HP:0007311, HP:000374...",0.268293
8,ENST00000542287,ATXN2,intron_variant,NaN,c.-65+602_-65+603insACA,chr12,111598973,-,TGT,NaN,...,NaN,NaN,6311.0,NaN,Q99700,rs769170503,0.815853,ATXN2,"[HP:0001151, HP:0002495, HP:0007311, HP:000374...",0.268293
15,ENST00000622544,GCH1,intron_variant,NaN,c.344-15065_344-15051del,chr14,54880487,ATATATATATACTCC,-,NaN,...,NaN,NaN,2643.0,NaN,P30793,rs1161730704,1.000000,GCH1,"[HP:0002487, HP:0003785, HP:0003781, HP:000245...",0.268293


In [10]:
ditto.to_csv("../../data/interim/UDN249098_HD.csv", index=False)